# Dappier

[Dappier](https://dappier.com) 将任何 LLM 或您的 Agentic AI 连接到来自可信来源的实时、已获授权的专有数据，使您的 AI 在任何领域都成为专家。我们的专业模型包括实时网络搜索、新闻、体育、金融股市数据、加密货币数据以及来自优质出版商的独家内容。请在我们的 marketplace.dappier.com 市场探索各种数据模型。

[Dappier](https://dappier.com) 提供经过优化、易于集成到 LangChain 的丰富、即时响应且具有情境相关性的数据字符串。无论您是构建会话式 AI、推荐引擎还是智能搜索，Dappier 的 LLM-agnostic RAG 模型都能确保您的 AI 访问经过验证的最新数据——无需您自行构建和管理检索流程的复杂性。

# DappierRetriever

这将帮助您开始使用 Dappier 的 [retriever](https://python.langchain.com/docs/concepts/retrievers/)。有关 DappierRetriever 所有功能和配置的详细文档，请访问 [API 参考](https://python.langchain.com/en/latest/retrievers/langchain_dappier.retrievers.Dappier.DappierRetriever.html)。

### 设置

安装 ``langchain-dappier`` 并设置环境变量 ``DAPPIER_API_KEY``。

```bash
pip install -U langchain-dappier
export DAPPIER_API_KEY="your-api-key"
```

我们还需要设置 Dappier API 凭据，这些凭据可以在 [Dappier 网站](https://platform.dappier.com/profile/api-keys) 上生成。

我们可以在 [Dappier 市场](https://platform.dappier.com/marketplace) 上查找支持的数据模型。

如果您想从单个查询中获取自动化跟踪，您还可以通过取消注释以下内容来设置您的 [LangSmith](https://docs.smith.langchain.com/) API 密钥：

In [ ]:
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
# os.environ["LANGSMITH_TRACING"] = "true"

### 安装

此检索器位于 `langchain-dappier` 包中：

In [ ]:
%pip install -qU langchain-dappier

## 实例化

- data_model_id: str
    数据模型 ID，以 `dm_` 开头。
    您可以在此处找到可用的数据模型 ID：
    [Dappier 市场。](https://platform.dappier.com/marketplace)
- k: int
    要返回的文档数量。
- ref: Optional[str]
    显示 AI 推荐的网站域名。
- num_articles_ref: int
    指定的 `ref` 域名文章的最小数量。
    其余文章将来自 RAG 模型中的其他网站。
- search_algorithm: Literal[
    "most_recent",
    "most_recent_semantic",
    "semantic",
    "trending"
]
    用于检索文章的搜索算法。
- api_key: Optional[str]
    用于与 Dappier API 交互的 API 密钥。

In [2]:
from langchain_dappier import DappierRetriever

retriever = DappierRetriever(data_model_id="dm_01jagy9nqaeer9hxx8z1sk1jx6")

## 用法

In [3]:
query = "latest tech news"

retriever.invoke(query)

[Document(metadata={'title': 'Man shot and killed on Wells Street near downtown Fort Wayne', 'author': 'Gregg Montgomery', 'source_url': 'https://www.wishtv.com/news/indiana-news/man-shot-dies-fort-wayne-december-25-2024/', 'image_url': 'https://images.dappier.com/dm_01jagy9nqaeer9hxx8z1sk1jx6/fort-wayne-police-department-vehicle-via-Flickr_.jpg?width=428&height=321', 'pubdata': 'Thu, 26 Dec 2024 01:00:33 +0000'}, page_content='A man was shot and killed on December 25, 2024, in Fort Wayne, Indiana, near West Fourth and Wells streets. Police arrived shortly after 6:30 p.m. following reports of gunfire and found the victim in the 1600 block of Wells Street, where he was pronounced dead. The area features a mix of businesses, including a daycare and restaurants.\n\nAs of the latest updates, police have not provided details on the safety of the area, potential suspects, or the motive for the shooting. Authorities are encouraging anyone with information to reach out to the Fort Wayne Police

## 在链中使用

与其它检索器一样，DappierRetriever 可以通过 [链](/docs/how_to/sequence/) 被整合到 LLM 应用中。

我们需要一个 LLM 或聊天模型：

In [6]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

In [7]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

prompt = ChatPromptTemplate.from_template(
    """Answer the question based only on the context provided.

Context: {context}

Question: {question}"""
)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [8]:
chain.invoke(
    "What are the key highlights and outcomes from the latest events covered in the article?"
)

"The key highlights and outcomes from the latest events covered in the article include:\n\n1. An Israeli airstrike in Gaza killed five journalists from Al-Quds Today Television, leading to condemnation from their outlet and raising concerns about violence against media professionals in the region.\n2. The Committee to Protect Journalists reported that since October 7, 2023, at least 141 journalists have been killed in the region, marking the deadliest period for journalists since 1992, with the majority being Palestinians in Gaza.\n3. A man was shot and killed in Fort Wayne, Indiana, with police not providing details on suspects, motive, or the safety of the area.\n4. An Oregon house cat died after eating pet food contaminated with the H5N1 bird flu virus, leading to a nationwide recall of Northwest Naturals' Feline Turkey Recipe raw frozen pet food and raising concerns about the spread of bird flu among domestic animals."

## API 参考

有关 DappierRetriever 所有功能和配置的详细文档，请前往 [API 参考](https://python.langchain.com/en/latest/retrievers/langchain_dappier.retrievers.Dappier.DappierRetriever.html)。